In [1]:
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# import libraries
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd

### Spotify API

In [3]:
# Set up API credentials
client_id = "3ff1e45f1c8d4189b317e02b3dc349b6"
client_secret = "878f0386a88f4c08b54199f30770c950"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

### Get Tracks

In [4]:
# Get a list of n tracks each year from start to end year (inclusive)
def top_tracks_by_year(start, end, top_n):
    results_by_year = {}

    for year in range(start, end+1):
        query = f'year:{year}'
        track_info = sp.search(q=query, type='track', limit=top_n, market='US')
        top_tracks = []

        for t in track_info['tracks']['items']:
            track_name = t['name']
            artists = t['artists'][0]['name']
            album = t['album']['name']
            popularity = t['popularity']
            track_id = t['id']
            
            top_tracks.append({
                'name': track_name,
                'artists': artists,
                'album': album,
                'popularity': popularity,
                'id': track_id
            })

        results_by_year[year] = top_tracks

    return results_by_year

# Change year and top n tracks here
top_tracks = top_tracks_by_year(1951, 2020, 20)

### Convert to DF

In [5]:
data = []

# Append tracks and their features to data
for year in top_tracks:
    tracks = top_tracks[year]
    rank = 1

    for track in tracks:
        data.append({
            'Year': year,
            'Track Name': track['name'],
            'Artist(s)': track['artists'],
            'Album': track['album'],
            'Popularity': track['popularity'],
            'Track ID': track['id']
        })
        rank += 1

# Create the DataFrame
df = pd.DataFrame(data)

# Create csv file, move this down after adding genre to df
### Delete this file each time you add new things to df, so it updates the file
df.to_csv('spotify_top_tracks_by_year.csv', index=False)  